<a href="https://colab.research.google.com/github/riccardocorradi/Trading-Room-Workshop/blob/main/Server_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The below code is the same as the one above except it also sends messages every 5 seconds. This causes all clients to cancel and replace all limit orders every 5 seconds ALONGSIDE every time there's new news. Could be beneficial if we are getting both our buys and sells filled before new news comes out, would want to keep placing orders at ub and lb to "market make" in the meantime. Also would trigger the start of our algorithm faster if price goes outside our range, we won't be waiting for new news to set the orders. Just update the time.sleep(5) to whatever amount of seconds you want. It also Locks threading, which may make communication speed a bit faster.

Current (should be final) server code:

In [ ]:
# STATE OF THE ART SERVER CODE FINAL

import socket
import threading
import time

estimates = {}
client_list = []
ticker_bounds = {       # Dictionaries to hold lower and upper bounds for each ticker
    "UB": {"lowerbounds": [], "upperbounds": []},
    "GEM": {"lowerbounds": [], "upperbounds": []}
}

haircut = .995

# Global variables to store the latest messages
message = ""
message1 = ""

def broadcast_periodic_updates():
    # Send combined message to all clients every 5 seconds
    global message, message1
    while True:
        if message and message1:
            combined_message = f"{message}\n{message1}"
            with threading.Lock():
                clients = client_list[:]  # Copy client_list outside the lock
            for client in clients:
                try:
                    client.send(combined_message.encode())
                except Exception as e:
                    print(f"Failed to send periodic message to client: {e}")
        time.sleep(5)

# Function to handle each client connection
def handle_client(client_socket, address):
    global message, message1
    print(f"Connection established with {address}")

    # Add client to client list
    with threading.Lock():
        client_list.append(client_socket)
    print("Client List: ", client_list)
    print(" ")

    while True:     # main loop to send and receive information with clients
        try:
            # Receive data from the client
            data = client_socket.recv(1024).decode()

            if not data:  # In case the client disconnects abruptly
                break

            # If the client sends 'exit', close the connection
            if data.lower() == 'exit':
                print(f"Client {address} disconnected.")
                break

            # Print and parse the received data
            print(f"Received from client: {data}")

            # Parse the message format "ticker;tick-estimate"
            if ";" in data:
                ticker, tick_estimate = data.split(";")
                message_list = tick_estimate.split("-")
            else:
                message_list = data.split("-")  # Default format for tick-estimate messages

            # Ensure message_list has expected length, else error.
            if len(message_list) == 2:
                # Parse tick and estimate values
                time = message_list[0]
                est = float(message_list[1])
                estimates[time] = est

                # Calculate lower and upper bounds based on the estimate
                lb = est - haircut*(299 - int(time)) / 50
                ub = est + haircut*(299 - int(time)) / 50

                # Ensure the ticker exists in the bounds dictionary
                if ticker not in ticker_bounds:
                    ticker_bounds[ticker] = {"lowerbounds": [], "upperbounds": []}

                # Append bounds if they fall within valid range
                ticker_bounds[ticker]["lowerbounds"].append(lb)
                ticker_bounds[ticker]["upperbounds"].append(ub)

                # Check if bounds lists are non-empty before calculating max/min
                lowerbounds = ticker_bounds[ticker]["lowerbounds"]
                upperbounds = ticker_bounds[ticker]["upperbounds"]

                if lowerbounds and upperbounds:
                    # Calculate the best lower and upper bounds
                    bottom = round(max(lowerbounds), 2)
                    top = round(min(upperbounds), 2)

                    # Define the message to send to the traders
                    message = f"{ticker} should be between ${bottom} and ${top}_"  # added a delimiter
                    print(message)
                    print("")

                    if ticker_bounds["UB"]["lowerbounds"] and ticker_bounds["GEM"]["upperbounds"]:
                        etf_bottom = round(max(ticker_bounds["GEM"]["lowerbounds"]) + max(ticker_bounds["UB"]["lowerbounds"]), 2)
                        etf_top = round(min(ticker_bounds["GEM"]["upperbounds"]) + min(ticker_bounds["UB"]["upperbounds"]), 2)

                        # Keep ETF range within case limits
                        if etf_bottom <= 60:
                            etf_bottom = 60.01
                        if etf_top >= 90:
                            etf_top = 89.99

                        message1 = f"ETF should be between ${etf_bottom} and ${etf_top}_"

                    # Send combined message to all connected traders immediately
                    combined_message = f"{message}\n{message1}"
                    with threading.Lock():  # Ensures thread safety when accessing client list
                        for client in client_list:
                            try:
                                client.send(combined_message.encode())
                            except Exception as e:
                                print(f"Failed to send message to client: {e}")
                else:
                    print(f"No valid bounds to calculate range for {ticker}.")

            else:
                print("Invalid message format received.")

        except ConnectionResetError:
            print(f"Client {address} disconnected abruptly.")
            break
        except ValueError as e:
            print(f"Value error: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")
            break

    # Clean up: close the connection and remove client from list
    with threading.Lock():
        client_list.remove(client_socket)
    client_socket.close()
    print(f"Connection closed with {address}")


def start_server():
    # Create a socket object
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # Bind the socket to the server's IP address and a port
    host = socket.gethostbyname(socket.gethostname())  # Get IP address dynamically
    port = 11111  # Keep the port the same

    server_socket.bind((host, port))  # Bind the new socket to a tuple

    # Start listening for incoming connections
    server_socket.listen(5)
    print(f"Server is listening on {host}:{port}...")

    # Start the periodic update thread
    periodic_thread = threading.Thread(target=broadcast_periodic_updates, daemon=True)
    periodic_thread.start()

    while True:
        # Accept a connection from a client
        client_socket, address = server_socket.accept()

        # Start a new thread to handle the client
        client_handler = threading.Thread(target=handle_client, args=(client_socket, address))
        client_handler.start()


# Start the server
start_server()